In [1]:
import numpy as np
import cv2
%matplotlib notebook
from matplotlib import pyplot as plt
import copy
import h5py
import numpy as np  
import tifffile
from skimage import io
import os
import sys
from tqdm import tqdm
import scipy
import time
import shutil

In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [3]:
def image_show(image, mymax, nrows=1, ncols=1, cmap='gray',size = 8):
    fig, ax = plt.subplots(nrows=nrows, ncols=ncols, figsize=(size, size*image.shape[0]/image.shape[1]))
    try:
        mymax
        ax.imshow(image, vmax = mymax, cmap='gray')
    except NameError:
        ax.imshow(image, cmap='gray')
    ax.axis('off')
    return fig, ax

## Format en face OTLS images ("slides") for deep learning algorithm (RGB)

In [7]:
path = 'images//raw images//normal//'
save_path = 'images//normal//'
for file in os.listdir(path):
    nuc = io.imread(path + file).astype('uint16')
    cyto_name = 'cyto_' + str(file[4:])
    cyto = io.imread(path + cyto_name).astype('uint16')

    ## Initialize image (ch0 in R, ch1 in G, nothing in B)
    im = np.zeros((nuc.shape[0],nuc.shape[1],3), dtype = 'uint16')
    im[:,:,0] = nuc
    im[:,:,1] = cyto
    tifffile.imwrite(save_path + str(file[4:]),im,photometric = 'rgb')

In [5]:
path = 'images//raw images//metaplasia//'
save_path = 'images//metaplasia//'
for file in os.listdir(path):
    nuc = io.imread(path + file).astype('uint16')
    cyto_name = 'cyto_' + str(file[4:])
    cyto = io.imread(path + cyto_name).astype('uint16')

    ## Initialize image (ch0 in R, ch1 in G, nothing in B)
    im = np.zeros((nuc.shape[0],nuc.shape[1],3), dtype = 'uint16')
    im[:,:,0] = nuc
    im[:,:,1] = cyto
    tifffile.imwrite(save_path + str(file[4:]),im,photometric = 'rgb')

## Write pyramidal files in 8-bit format

In [8]:
#code adapted from https://forum.image.sc/t/standard-workflow-and-format-to-handle-huge-slidescanner-images/28159/20
path = 'images//normal//'

for file in os.listdir(path):
    if file.endswith('.tiff'):
        image = tifffile.imread(path + file, key=0).astype('float64')

        ## Scale channel 0 to 0-255
        image[:,:,0] = image[:,:,0]/np.max(image[:,:,0])
        image[:,:,0] = image[:,:,0]*255
        ## Scale channel 1 to 0-255
        image[:,:,1] = image[:,:,1]/np.max(image[:,:,1])
        image[:,:,1] = image[:,:,1]*255
        ## Set as uint8
        image = image.astype('uint8')
        
        h, w, s = image.shape
        img_resolution = .209 #um/px
        px_per_cm = 10000/img_resolution #inverse of image resolution in cm/px
        level = 0

        with tifffile.TiffWriter(path + file[:-1], bigtiff=True) as tif: #we want the file to safe as .tif, not .tiff
            while True:    
                options = dict(tile = (int(256/2**level),int(256/2**level)))
                tif.write(image, software='Glencoe/Faas pyramid', metadata=None,
                    resolution=(px_per_cm/(2**level), px_per_cm/(2**level), 'CENTIMETER'),
                    **options)
                tif.write(image[::2,::2,:], subfiletype=1, **options)
                if max(w, h) < 256 or level >= 4: #Not sure why it only works for levels 1-4. should troubleshoot this later
#                     print(level)
                    break
                level += 1
                w //= 2
                h //= 2
                image = cv2.resize(image, dsize=(w, h), interpolation=cv2.INTER_LINEAR)

In [9]:
#code adapted from https://forum.image.sc/t/standard-workflow-and-format-to-handle-huge-slidescanner-images/28159/20
path = 'images//metaplasia//'

for file in os.listdir(path):
    if file.endswith('.tiff'):
        image = tifffile.imread(path + file, key=0).astype('float64')
#         image = io.imread(path + file).astype('float64')
        ## Scale channel 0 to 0-255 (without clipping)
        image[:,:,0] = image[:,:,0]/np.max(image[:,:,0])
        image[:,:,0] = image[:,:,0]*255
        ## Scale channel 1 to 0-255 (without clipping)
        image[:,:,1] = image[:,:,1]/np.max(image[:,:,1])
        image[:,:,1] = image[:,:,1]*255
        ## Set as uint8
        image = image.astype('uint8')        
        
        h, w, s = image.shape
        img_resolution = .209 #um/px
        px_per_cm = 10000/img_resolution #inverse of image resolution in cm/px
        level = 0

        with tifffile.TiffWriter(path + file[:-1], bigtiff=True) as tif: #we want the file to safe as .tif, not .tiff
            while True:    
                options = dict(tile = (int(256/2**level),int(256/2**level)))
                tif.write(image, software='Glencoe/Faas pyramid', metadata=None,
                    resolution=(px_per_cm/(2**level), px_per_cm/(2**level), 'CENTIMETER'),
                    **options)
                tif.write(image[::2,::2,:], subfiletype=1, **options)
                if max(w, h) < 256 or level >= 4: #Not sure why it only works for levels 1-4. should troubleshoot this later
#                     print(level)
                    break
                level += 1
                w //= 2
                h //= 2
                image = cv2.resize(image, dsize=(w, h), interpolation=cv2.INTER_LINEAR)